In [1]:
import numpy as np
import cv2
import glob
import pickle
from tqdm import tqdm

In [2]:
camera = "diegos_iriun"
imgs_path = "cameras/" + camera
format = "png"
square_size = 0.0235
chess_width = 9
chess_height = 6

In [3]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

def calibrate(dirpath, image_format, square_len, width=9, height=6):
    """ Apply camera calibration operation for images in the given directory path. """
    
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(8,6,0)
    objp = np.zeros((height*width, 3), np.float32)
    objp[:, :2] = np.mgrid[0:width, 0:height].T.reshape(-1, 2)
    objp = objp * square_len 
    
    # Arrays to store object points and image points from all the images.
    objpoints = []  # 3d point in real world space
    imgpoints = []  # 2d points in image plane.
    
    images = glob.glob(dirpath+'/' + '*.' + image_format)
    print(f"{len(images)} found")
    
    img = cv2.imread(images[0])
    (calib_h, calib_w) = img.shape[:2]
    print(f"Resolution: {calib_w}x{calib_h}")

    for fname in tqdm(images):
    
        img = cv2.imread(fname)

        if img.shape[:2] != (calib_h, calib_w):
            print("Calibration pictures have different resolutions!")


        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # print(gray.shape[::-1])
        
        # Find the chess board corners
        ret, corners = cv2.findChessboardCorners(gray, (width, height), None)
        
        # If found, add object points, image points (after refining them)
        if ret:
            objpoints.append(objp)
            corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            imgpoints.append(corners2)
            
            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, (width, height), corners2, ret)
            # Resize image
            img_height, img_width = img.shape[:2]
            max_width = 1200
            max_height = 1080
            scale_factor = min(max_width / img_width, max_height / img_height)
            img = cv2.resize(img, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_AREA)
            cv2.imshow('Calibration image', img)
            cv2.waitKey(50)
    
    cv2.destroyAllWindows()

    return cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None), calib_w, calib_h

In [4]:
(ret, mtx, dist, rvecs, tvecs), calib_w, calib_h = calibrate(imgs_path, format, square_size, chess_width, chess_height)

32 found
Resolution: 3840x2160


100%|██████████| 32/32 [02:38<00:00,  4.94s/it]


In [5]:
# create a dictionary to store the calibration parameters
calibration_params = {"ret": ret, "mtx": mtx, "dist": dist, "rvecs": rvecs, "tvecs": tvecs, "calib_w": calib_w, "calib_h": calib_h}

# save the parameters in a pickle file
with open('cameras/' + camera + '/calibration_params.pickle', 'wb') as f:
    pickle.dump(calibration_params, f)